In [27]:
from wiktionaryparser import WiktionaryParser
import requests
from bs4 import BeautifulSoup as bs
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  
import pandas as pd
df = pd.read_csv('/home/jackragless/projects/local/DAIC_glossary_generator/big_data/unigram_freq.csv')
df = list(df['word'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
parser = WiktionaryParser()
def wikt_extract(kw):
    extract = parser.fetch(kw,'english')
    temp_def = {}
    for i in extract[0]['definitions']:
        temp_pos = []
        for j in i['text'][1:]:
            if not (j[0]=='(' and j[-1]==')'):
                temp_pos.append(j)
        temp_def[i['partOfSpeech']] = temp_pos
    return {kw:temp_def}

In [29]:
def get_domains(term):
    final = []
    for pos in list(wikt_extract(term)[term].keys()):
        for sent in wikt_extract(term)[term][pos]:
            if sent[0] == '(' and sent.find(')') != -1:
                domain = sent[:sent.find(')')].replace('(','').replace(')','')
                domain = domain.split(', ')
                final += domain
    return final

In [31]:
FINAL = []
FAILED = []
count = 0
for word in df[10000:12000]:
    count += 1
    if word not in stop_words:
        try:
            FINAL += get_domains(word)
        except: 
            FAILED.append(word)
        print(count, end='\r')

In [43]:
result = pd.Series(FINAL).value_counts().index.tolist()
result_freq = list(pd.Series(FINAL).value_counts())

In [48]:
data = {'result':pd.Series(FINAL).value_counts().index.tolist(), 'freq':list(pd.Series(FINAL).value_counts())}

In [50]:
pd.DataFrame(data).to_csv('wikt_domain_list.csv')